In [1]:
from huggingface_hub import snapshot_download
from txtai.workflow import Task
from txtai.workflow import Workflow
from txtai.pipeline import Tabular
from txtai.pipeline import Similarity
from txtai.embeddings import Embeddings
import pandas as pd
import gradio as gr
import numpy as np
import json
import os
os.environ['TRANSFORMERS_CACHE'] = 'cache'

try:
    from tqdm.auto import tqdm
except ImportError:
    def tqdm(x): return x


2023-05-04 17:52:45.133688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 17:52:45.672168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
rerun = False
filename = "ctgov"
ckptpath = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
indexfile = f'{filename}_{ckptpath.replace("/", "-")}.index'
snapshot_download(repo_id=ckptpath,
                  repo_type="model",
                  cache_dir="cache")
embeddings = Embeddings({
    "method": "transformers",
    "path": ckptpath,
    "content": True,
    "object": True
})

if os.path.exists(indexfile) and rerun is False:
    print("Indexed and Cached!")
    embeddings.load(indexfile)
else:
    print("Need to rerun or Indices and Caches dont exist, run them!")

    # Create tabular instance mapping input.csv fields
    tabular = Tabular(idcolumn="nct_id", content=True)

    # Create workflow
    workflow = Workflow([Task(tabular)])

    # Indexinh the data
    data = list(workflow([f'{filename}.csv']))
    embeddings.index(tqdm(data))
    embeddings.save(indexfile)
    print("Indexing and Caching finished for the 1st time!")


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

Need to rerun or Indices and Caches dont exist, run them!


  0%|          | 0/540916 [00:00<?, ?it/s]

Indexing and Caching finished for the 1st time!
